Tutorial: https://www.youtube.com/watch?v=q2ZOEFAaaI0&t=252s

Taxiv2 based on Q_Learning that is based on Exploration and Exploitation Model 

In [1]:
import numpy as np 
import gymnasium as gym
import random 

In [2]:
env = gym.make("Taxi-v3", render_mode="human")

In [3]:
env.reset()

(346, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [ ]:
# Testing environment 
episodes = 1
for episode in range(1, episodes+1):
    done = False
    score = 0
    state = env.reset()

    while not done:
        env.render()
        action = env.action_space.sample()
        state, reward, done, truncated, info = env.step(action)
        score += reward
        print("\r state:{} reward:{} done:{} info:{} ".format(state, reward, done, info))
    print("Epi:{} score:{}".format(episode, score))


 state:64 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 0, 1, 1, 0, 0], dtype=int8)} 
 state:64 reward:-10 done:False info:{'prob': 1.0, 'action_mask': array([1, 0, 1, 1, 0, 0], dtype=int8)} 
 state:164 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)} 
 state:264 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)} 
 state:284 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)} 
 state:284 reward:-10 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)} 
 state:284 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)} 
 state:264 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)} 
 state:164 reward:-1 done:False info:{'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)} 
 state:144 reward:-1 done:False info:{'prob': 

# Q-table 

In [4]:
# action_size and state_size
action_size = env.action_space.n
obs_size = env.observation_space.n
print(action_size, obs_size, env.observation_space)

6 500 Discrete(500)


In [5]:
Q_table = np.zeros((obs_size, action_size))

In [9]:
# hyperparameters 
total_episodes = 500000
total_test_episodes = 100
max_steps = 99

learning_rate = 0.7
gamma = 0.618

#Exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.001
decay_rate = 0.001


In [10]:
state = env.reset()
print(state)
state.__getitem__(0)

(6, {'prob': 1.0, 'action_mask': array([1, 0, 1, 0, 0, 0], dtype=int8)})


6

In [11]:
action = env.action_space.sample()

# after taking action, observe new_state and reward 
new_state, reward, done, truncated, info = env.step(action)

In [12]:
new_state

6

# Exploration and Exploitation model and Updating Q_table

In [14]:
# learing model 

for episode in range(total_episodes):
    done = False
    step = 0 
    state = env.reset()
    
    for step in range(max_steps):
        # print("----------top:{}---------".format(step))
        # action from current state
        # first randomize 
        exp_exp_tradeoff = random.uniform(0,1)
        
        if exp_exp_tradeoff > epsilon: # do exploitation ie taking biggest Q-value for this state
            action = np.argmax(Q_table[state[0], :])
        else: # do random choice : do exploration randomly the env
            action = env.action_space.sample()

        # after taking action, observe new_state and reward 
        new_state, reward, done, truncated, info = env.step(action)

        # update Q_value of Q_table : Bellman equation
        Q_table[state[0], action] = Q_table[state[0], action] + learning_rate*(reward + gamma * np.max(Q_table[new_state, :]) - Q_table[state[0],action])
        
        # update current obs_state
        state = new_state

        if done == True: #breaking if taxi did
            state = env.reset()
            print("Episode:{}  Epsilon: {}, done:{} \n".format(episode, epsilon, done)) 
            break
        
        episode += 1

        # reduce epsilon for less and less exploration and more exploitation 
        epsilon = min_epsilon + (max_epsilon - min_epsilon) *  np.exp(-decay_rate * episode)
        
        # print("----------end:{}---------".format(step))

print("Model Completed !! \n Epsilon: {} \n Q_Table:{}".format(epsilon, Q_table))
          
            
        
        

TypeError: 'int' object is not subscriptable

In [15]:
0.4634113110390152

0.4634113110390152

In [16]:
# trained Q_values count, q_count
Q_count = 0
for i in range(env.observation_space.n):
    trainedRowSum = sum(Q_table[i,:])
    if trainedRowSum != float(0):
        print("state: {} ".format(i), Q_table[i,:], "Sum: {}".format(trainedRowSum))
        Q_count += 1 
        
print("\n\n--> The total trained Q_values are : {}\n\n".format(Q_count))

state: 229  [ 0.   0.  -0.7  0.   0.   0. ] Sum: -0.7
state: 341  [-0.7  0.   0.   0.   0.   0. ] Sum: -0.7


--> The total trained Q_values are : 2




In [ ]:
for i in range(obs_size):
    print("state: {} ".format(i), Q_table[i,:])

In [ ]:
state = env.reset()
state

In [ ]:
state[0]

In [ ]:
Q_table[state[0], :]

In [ ]:
action = np.argmax(Q_table[state[0],:])
action

In [ ]:
# Playing with eplore and expoit 

rewards = []
for episode in range(int(0.1*total_test_episodes)):
    
    done = False 
    step = 0
    score = 0
    print("-------------------------------------------------------\n", "Episode: {}".format(episode), "\n")
    for step in range(int(0.2*max_steps)):
        env.render()
        print(state)
        state_i=state[0]
        action = np.argmax(Q_table[state_i,:]) # action using Q_table as pie-policy/ state-action policy
        
        new_state, reward, done , truncated, info = env.step(action)
        score += reward # total_rewards

        if done:
            state = env.reset()
            rewards.append(score)
            print("Score: {}".format(score))
            break

        state = new_state

env.close()
print("Score over time: {}".format((sum(rewards)/total_test_episodes)))
            
        
        
        
        